# Top Three Salaries

As part of an ongoing analysis of salary distribution within the company, your manager has requested a report identifying high earners in each department. A 'high earner' within a department is defined as an employee with a salary ranking among the top three salaries within that department.

You're tasked with identifying these high earners across all departments. Write a query to display the employee's name along with their department name & salary. In case of duplicates sort the results of department name in ascending order, then by salary in descending order. If multiple employees have the same salary, then order them alphabetically.

Note: Utilise the appropriate ranking window function to handle duplicate salaries effectively.

# Answer

We'll be reusing the `employee2` table:

```
SELECT * FROM employee2;
```

<img src = "employee2 Table.png" width = "600" style = "margin:auto"/>

I wonder what the note means... I'll rank all employee salaries by department before filtering to the top three.

```
SELECT *,
	   rank() OVER (PARTITION BY department_id
	   	   ORDER BY salary DESC)
FROM employee2
ORDER BY department_id, salary DESC;
```

<img src = "Salaries Ranked by Department.png" width = "600" style = "margin:auto"/>

Ok, good thing I used the `rank()` function. There is a two-way tie for 2nd highest paid employee in department 1, & the `rank()` function appropriately continues with the rank 4 instead of 3 (`dense_rank()` would have outputted 3 for the 4th highest paid employee), allowing us to use a simple number filter to find the top 3 salaries in each department. 

```
SELECT *
FROM (
	SELECT *,
		   rank() OVER (PARTITION BY department_id
		   	   ORDER BY salary DESC)
	FROM employee2
	ORDER BY department_id, salary DESC, name
)
WHERE rank <= 3;
```

<img src = "Top 3 Salaries by Department.png" width = "600" style = "margin:auto"/>

Oh! I almost forgot. Here is the `department` table, so we can understand what each department is. We'll also join it to our query.

```
SELECT department.department_name,
	   dep_sal.name,
	   dep_sal.salary,
	   dep_sal.rank
FROM (
	SELECT *,
		   rank() OVER (PARTITION BY department_id
		   	   ORDER BY salary DESC)
	FROM employee2
	ORDER BY department_id, salary DESC, name
) AS dep_sal
LEFT JOIN department
	ON dep_sal.department_id = department.department_id
WHERE dep_sal.rank <= 3;
```

<img src = "Top 3 Salary For Each Department.png" width = "600" style = "margin:auto"/>

In the Data Analytics department, Olivia Smith leads with a salary of \\$7000, followed by a two-way tie between Amelia Lee & James Anderson with salaries of \\$4000. In the Data Science department, Logan Moore leads with a salary of \\$8000, followed by Charlotte Miller with a salary of \\$7000, which is further followed by a two-way tie for third highest salary of \\$6800 between Noah Johnson & William Davis. In the Data Engineering department, Liam Brown has the highest salary of \\$13000, Ava Garcia has the second highest salary of \\$12500, & Isabella Wilson has the third highest salary of \\$11000.